In [1]:
from agents import Agent, WebSearchTool, ModelSettings
from dotenv import load_dotenv
from planner_agent import WebSearchItemList, WebSearchItem, plan_deep_search
import asyncio

load_dotenv(override=True)
VALIDATOR_INSTRUCTION = "You are a validation assistant. Given a search term and a generated summary, your task is to verify whether the summary adheres to the INSTRUCTIONS. The summary should be 2–3 paragraphs, under 300 words, concise, and focused only on the essential points from the web search results. Writing should be compact, possibly using fragments or short phrases, and free of commentary, filler, or unnecessary detail. The output must contain only the summary itself, no explanations or formatting. You will assess compliance and quality. If the summary violates any rule, briefly describe the issues. Always output your evaluation in three parts: Compliance (Yes/No), Issues (if any), and Quality Score (0–10). Be objective, professional, and concise in your assessment."


search_agent_validator = Agent(
    name="Search agent validator",
    instructions = VALIDATOR_INSTRUCTION,
    model = "gpt-4o-mini"
)

search_validator_tool = search_agent_validator.as_tool(tool_name="search_validator", tool_description="Validates whether a generated web search summary complies with the defined formatting and content INSTRUCTIONS. "
    "It checks that the summary is 2–3 paragraphs, under 300 words, concise, and focused only on key findings without commentary or filler. "
    "The tool outputs an objective evaluation including compliance status (Yes/No), a brief description of any issues, "
    "and a quality score from 0–10 based on clarity, conciseness, and adherence to the guidelines.")

In [2]:
INSTRUCTIONS = (
    "You are a research assistant. Given a search term, you search the web for that term and "
    "produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 "
    "words. Capture the main points. Write succintly, no need to have complete sentences or good "
    "grammar. This will be consumed by someone synthesizing a report, so its vital you capture the "
    "essence and ignore any fluff. Do not include any additional commentary other than the summary itself."
    "After producing the summary (and just after producing the summary), always use the search_validator tool to check compliance with the INSTRUCTIONS before finalizing your answer."
)

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low"), search_validator_tool],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [12]:
#from asyncio import Runner, as_completed
from sys import exception
from agents import Runner  


async def search(item: WebSearchItem):
    input = f"Search Search term:: {item.query}\n, Reason for searching: {item.reason}"
    result = await Runner.run(
        search_agent,
        input
    )
    return str(result.final_output)


async def perform_web_search(webSearchItemList: WebSearchItemList):
    print("Searching...")
    num_completed = 0
    tasks = [asyncio.create_task(search(item)) for item in webSearchItemList.searches]
    results = []
    for task in asyncio.as_completed(tasks):
        result = await task
        if result is not None:
            results.append(result)
        num_completed += 1
        print(f"Searching... {num_completed}/{len(tasks)} completed")
    print("Finished searching")
    return results


query_list = await plan_deep_search("What are the most usefull frameworks for developing AI Chatebot that help  IT help-desks solves their daily Tasks")
print(query_list.searches[0])
print(await search(query_list.searches[0]))
        


Planning searches...
generating a new Item
Regeneration of the prompt for the keyword result: case studies AI chatbots IT help-desk
reason='To find a current list of AI chatbot frameworks specifically suited for IT help-desk applications.' query='best AI chatbot frameworks for IT help-desk'
Several AI chatbot frameworks are well-suited for IT help-desk applications, offering features like automation, integration capabilities, and user-friendliness. **Botpress** is an open-source platform providing a comprehensive toolkit for designing, training, and deploying AI-driven support bots. It allows for integration with popular AI models and incorporates rule-based logic to manage inquiries and answer from a knowledge base effectively.

**Dante** simplifies customer support with no-code deployment, enabling custom AI agents that can handle FAQs and operate 24/7 across various channels like websites, Facebook Messenger, and WhatsApp. **Zendesk** features an AI-driven chatbot that integrates wi

In [10]:
print(query_list.searches[0])
print(await search(query_list.searches[0]))

reason='To find a comprehensive list of frameworks designed specifically for AI chatbots, especially in the context of IT help-desks.' query='best AI chatbot frameworks for IT help-desk'
The summary of AI chatbot frameworks for IT help-desks is compliant with the guidelines and has a quality score of 9.


In [13]:
results = await perform_web_search(query_list)
print(results)

Searching...
Searching... 1/3 completed
Searching... 2/3 completed
Searching... 3/3 completed
Finished searching
['For developing AI chatbots tailored for IT help-desk automation, several frameworks stand out:\n\n- **Botpress**: Open-source platform with a visual studio for conversation design, integrating AI models like OpenAI’s GPT-4, and supporting rule-based logic. Facilitates handling inquiries, accessing knowledge bases, and performing guided workflows.\n\n- **Dialogflow CX (Google Cloud)**: Features a state machine-based flow builder for complex conversations, integrates with Google Cloud’s AI models, and supports multi-language. Offers built-in speech-to-text and text-to-speech capabilities.\n\n- **Microsoft Bot Framework with Azure AI**: Code-first framework providing control over chatbot logic, integrates with Azure AI for natural language processing, and offers Bot Composer for low-code development.\n\n- **Rasa**: Open-source framework with components for natural language un